<a href="https://colab.research.google.com/github/GundetiManoj99/GAN/blob/main/GANs_for_image_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# Ensure the directory exists
os.makedirs('./results', exist_ok=True)

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
from torch.autograd import Variable

batchSize = 64
imageSize = 64

transform = transforms.Compose([transforms.Resize(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), ])

dataset = dset.CIFAR10(root='./data', train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batchSize,
    shuffle=True, num_workers=2)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

class G(nn.Module):

    def __init__(self):
        super(G,self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(100,512,4,1,0,bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512,256,4,2,1,bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256,128,4,2,1,bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128,64,4,2,1,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64,3,4,2,1,bias=False),
            nn.Tanh()
        )

    def forward(self,input):
        output = self.main(input)
        return output

netG = G()
netG.apply(weights_init)

class D(nn.Module):
    def __init__(self):
        super(D,self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3,64,4,2,1,bias=False),
            nn.LeakyReLU(0.2,inplace=True),

            nn.Conv2d(64,128,4,2,1,bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2,inplace=True),

            nn.Conv2d(128,256,4,2,1,bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2,inplace=True),

            nn.Conv2d(256,512,4,2,1,bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2,inplace=True),

            nn.Conv2d(512,1,4,1,0,bias=False),
            nn.Sigmoid()
        )

    def forward(self,input):
        output = self.main(input)
        return output.view(-1)

netD = D()
netD.apply(weights_init)

#training the DCGANs
criterion = nn.BCELoss()
optimizerD=optim.Adam(netD.parameters(),lr=0.0002,betas=(0.5,0.999))
optimizerG=optim.Adam(netG.parameters(),lr=0.0002,betas=(0.5,0.999))

for epoch in range(25):
    for i,data in enumerate(dataloader,0):
        netD.zero_grad()

        real,_ = data
        input = Variable(real)
        target = Variable(torch.ones(input.size()[0]))

        output = netD(input)
        errD_real = criterion(output,target)

        noise= torch.randn(input.size()[0],100,1,1)
        fake = netG(noise)
        target = Variable(torch.zeros(input.size()[0]))
        output=netD(fake.detach())
        errD_fake = criterion(output,target)

        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()

        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]))
        output = netD(fake)
        errG = criterion(output,target)
        errG.backward()
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f'
      % (epoch, 25, i, len(dataloader), errD.item(), errG.item()))


        if i % 100 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize=True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize=True)


Files already downloaded and verified
[0/25][0/782] Loss_D: 1.9561 Loss_G: 3.5374
[0/25][1/782] Loss_D: 1.7260 Loss_G: 5.6722
[0/25][2/782] Loss_D: 0.7783 Loss_G: 6.1428
[0/25][3/782] Loss_D: 0.8105 Loss_G: 5.0774
[0/25][4/782] Loss_D: 0.9128 Loss_G: 6.3546
[0/25][5/782] Loss_D: 0.8207 Loss_G: 6.3343
[0/25][6/782] Loss_D: 1.0429 Loss_G: 7.3977
[0/25][7/782] Loss_D: 0.8047 Loss_G: 7.1952
[0/25][8/782] Loss_D: 0.7260 Loss_G: 8.2272
[0/25][9/782] Loss_D: 0.8734 Loss_G: 7.1575
[0/25][10/782] Loss_D: 0.9650 Loss_G: 9.6688
[0/25][11/782] Loss_D: 0.5394 Loss_G: 7.4198
[0/25][12/782] Loss_D: 0.9489 Loss_G: 10.4453
[0/25][13/782] Loss_D: 0.3765 Loss_G: 8.1937
[0/25][14/782] Loss_D: 0.7240 Loss_G: 10.1110
[0/25][15/782] Loss_D: 0.5230 Loss_G: 8.3624
[0/25][16/782] Loss_D: 0.9508 Loss_G: 12.8917
[0/25][17/782] Loss_D: 0.2990 Loss_G: 9.9956
[0/25][18/782] Loss_D: 0.5405 Loss_G: 10.8091
[0/25][19/782] Loss_D: 0.3119 Loss_G: 9.2934
[0/25][20/782] Loss_D: 0.5496 Loss_G: 13.4103
[0/25][21/782] Loss_D: